In [12]:
import pandas as pd
import os
import sqlite3 as sql3

In [13]:
currentdir = os.getcwd()
db_path = f"{currentdir}\\S30 ETL Assignment.db"

In [14]:
conn = sql3.connect(db_path)
with conn:
    cur = conn.cursor()
    cur.execute("""
        WITH customer_sales AS (
            SELECT
                customers.customer_id,
                customers.age,
                sales_id
            FROM customers
            INNER JOIN sales
                ON sales.customer_id = customers.customer_id
            WHERE 
                customers.age >= 18
                AND
                customers.age <= 35
        )
        SELECT
            customer_id,
            age,
            item_name,
            SUM(CAST(quantity AS INTEGER)) as order_quantity
        FROM orders
        INNER JOIN customer_sales
            ON customer_sales.sales_id = orders.sales_id
        INNER JOIN items
            ON items.item_id = orders.item_id
        WHERE quantity > 0
        GROUP BY 1,2,3
        ORDER BY customer_id ASC, quantity DESC
    """)
    results = cur.fetchall()

In [15]:
# Used Pandas just for exporting to CSV
df = pd.DataFrame(results, columns=['customer_id','age','item_name','order_quantity'])
df.to_csv('test_python_sqlite3.csv', sep=';', index=False)